In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# from collections import defaultdict


# =Directories=

INPUT_DIR = 'in'
OUTPUT_DIR = 'out'
PLOT_DIR = f'{OUTPUT_DIR}/plots'
PLOT_PREDICTIONS_DIR = f'{OUTPUT_DIR}/plots/predictions'

os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(PLOT_DIR, exist_ok=True)
os.makedirs(PLOT_PREDICTIONS_DIR, exist_ok=True)

In [ ]:
df = pd.read_csv(f"{INPUT_DIR}/43311-0002_de.csv",  delimiter=';')


# Fill missing Jahr & Monat
df["Jahr"].fillna(method="ffill", inplace=True)
df["Monat"].fillna(method="ffill", inplace=True)
df["Jahr"] = df["Jahr"].astype(int)

# Convert production columns to numeric
df["Elektrizitätserzeugung (brutto)"] = pd.to_numeric(df["Elektrizitätserzeugung (brutto)"], errors='coerce')
df["Elektrizitätserzeugung (netto)"] = pd.to_numeric(df["Elektrizitätserzeugung (netto)"], errors='coerce')

# Group by Jahr and Typ 
strommix_df = df.groupby(["Jahr", "Typ"])[["Elektrizitätserzeugung (netto)"]].sum().reset_index()

strommix_df.head(20)

In [ ]:
import pandas as pd

# CSV einlesen
df_saldo = pd.read_csv(f"{INPUT_DIR}/43312-0002_de_flat.csv", sep=";")

# Filter: Deutschland insgesamt + Insgesamt
df_saldo_filtered = df_saldo[
    (df_saldo["2_variable_attribute_label"] == "Deutschland") &
    (df_saldo["3_variable_attribute_label"] == "Insgesamt") 
]

# Parse numbers
df_saldo_filtered["Jahr"] = df_saldo_filtered["time"].astype(int)
df_saldo_filtered["value"] = pd.to_numeric(
    df_saldo_filtered["value"]
    .astype(str)
    .str.replace(".", "", regex=False)
    .str.replace(",", ".", regex=False),
    errors="coerce"
)

# Group: Jahr + Typ und Werte summieren
grouped = (
    df_saldo_filtered
    .groupby(["Jahr", "value_variable_label"])["value"]
    .sum()
    .reset_index()
)

grouped = grouped.rename(columns={
    "value_variable_label": "Typ",
    "value": "Elektrizitätserzeugung (netto)"
})

# Timeframe: 2002–2024
austauschsaldo = grouped[(grouped["Jahr"] >= 2002) & (grouped["Jahr"] <= 2024)]

# austauschsaldo.head(20)

# Concat export data
strommix_gesamt = pd.concat([strommix_df, austauschsaldo], ignore_index=True)

# Test output
strommix_gesamt[strommix_gesamt["Jahr"] == 2022].sort_values("Jahr")

In [ ]:
# Wind / Solar energie import

import json

# Load JSON file
with open(f"{INPUT_DIR}/bruttostromerzeugung-energieträger.json", "r", encoding="utf-8") as f:
    brutto_data = json.load(f)

# Existing years in the database
valid_years = set(strommix_gesamt["Jahr"].unique())

# Extract only Windkraft and Photovoltaik entries
zusatz_daten = {
    "Windkraft": brutto_data.get("Wind", {}),
    "Photovoltaik": brutto_data.get("Photovoltaik", {})
}

# Collect valid records
records = []
for typ, jahresdaten in zusatz_daten.items():
    for jahr, value in jahresdaten.items():
        try:
            jahr_int = int(jahr)
            if jahr_int in valid_years and value and value.strip():
                value_float = float(value.replace(",", "."))
                records.append({
                    "Jahr": jahr_int,
                    "Typ": typ,
                    "Elektrizitätserzeugung (netto)": value_float * 1_000_000  # TWh -> MWh
                })
        except ValueError:
            continue  # skip malformed entries

df_zusatz = pd.DataFrame(records)

# Remove existing entries for these types to avoid duplicates and concat data
strommix_gesamt = strommix_gesamt[
    ~strommix_gesamt["Typ"].isin(["Windkraft", "Photovoltaik"])
]
strommix_gesamt = pd.concat([strommix_gesamt, df_zusatz], ignore_index=True)

# Json is missing 2024 data
zusatz_2024 = pd.DataFrame([
    {"Jahr": 2024, "Typ": "Windkraft", "Elektrizitätserzeugung (netto)": 138_900_000},
    {"Jahr": 2024, "Typ": "Photovoltaik", "Elektrizitätserzeugung (netto)": 74_100_000}
])

strommix_gesamt = pd.concat([strommix_gesamt, zusatz_2024], ignore_index=True)

# Check result
strommix_gesamt[strommix_gesamt["Typ"].isin(["Windkraft", "Photovoltaik"])].sort_values(["Typ", "Jahr"])



In [ ]:
# Post Processing

# Category mappping table
typ_to_category = {
    # Fossile Brennstoffe
    "Braunkohlenbriketts": "Braunkohle",
    "Braunkohlenkoks": "Braunkohle",
    "Rohbraunkohlen": "Braunkohle",
    "Sonstige Braunkohlen": "Braunkohle",
    "Hartbraunkohlen": "Braunkohle",

    "Steinkohlen": "Steinkohle",
    "Steinkohlenbriketts": "Steinkohle",
    "Steinkohlenkoks": "Steinkohle",
    "Kohlenwertstoffe aus Steinkohle": "Steinkohle",
    "Sonstige Steinkohlen": "Steinkohle",
    "Staub- und Trockenkohle": "Steinkohle",
    "Wirbelschichtkohle": "Steinkohle",

    "Dieselkraftstoff": "Mineralöl",
    "Heizöl, leicht": "Mineralöl",
    "Heizöl, schwer": "Mineralöl",
    "Petrolkoks": "Mineralöl",
    "Raffineriegas": "Mineralöl",
    "Sonstige Mineralölprodukte": "Mineralöl",

    "Erdgas, Erdölgas": "Erdgas",
    "Flüssiggas": "Erdgas",

    "Hochofengas": "Industriegas",
    "Kokereigas": "Industriegas",
    "Sonstige hergestellte Gase": "Industriegas",

    # Erneuerbare
    "Photovoltaik": "Solar",
    "Solarthermie": "Solar",

    "Laufwasser": "Wasserkraft",
    "Pumpspeicherwasser": "Wasserkraft",
    "Pumpspeicher mit natürlichem Zufluss": "Wasserkraft",
    "Pumpspeicher ohne natürlichen Zufluss": "Wasserkraft",
    "Speicherwasser": "Wasserkraft",

    "Windkraft": "Wind",

    "Feste biogene Stoffe": "Biomasse",
    "Flüssige biogene Stoffe": "Biomasse",
    "Biogas": "Biomasse",
    "Biomethan (Bioerdgas)": "Biomasse",
    "Deponiegas": "Biomasse",
    "Klärgas": "Biomasse",
    "Klärschlamm": "Biomasse",
    "Grubengas": "Biomasse",

    "Geothermie": "Geothermie",
    "Wärmepumpen (Erd- und Umweltwärme)": "Geothermie",

    "Sonstige erneuerbare Energien": "Andere Erneuerbare",

    # Abfall
    "Abfall (Hausmüll, Industrie)": "Abfall",
    "Abfall (Hausmüll, Siedlungsabfälle)": "Abfall",
    "Abfall (Industrie)": "Abfall",

    # import / export
    "Einfuhr von Elektrizität": "Import",
    "Ausfuhr von Elektrizität": "Export",


    # Sonstige
    "Kernenergie": "Kernenergie",

    "Austauschsaldo": "Import/Export",

    "Strom (Elektrokessel)": "Sonstiges",
    "Andere Speicher": "Sonstiges",
    "Wasserstoff": "Wasserstoff", # todo make it be it's own category
    "Wärme": "Sonstiges",
    "Sonstige Energieträger": "Sonstiges",

}

# strip Typ field
strommix_gesamt["Typ"] = strommix_gesamt["Typ"].str.strip()
# remove the 'Insgesamt' field 
strommix_gesamt = strommix_gesamt[strommix_gesamt["Typ"] != "Insgesamt"]


# Add the Kategorie field
strommix_gesamt["Kategorie"] = strommix_gesamt["Typ"].map(typ_to_category).fillna("Unbekannt")

# Unbekannte debugging
unbekannte_typen = strommix_gesamt[strommix_gesamt["Kategorie"] == "Unbekannt"]["Typ"].unique()
print(f"Unbekannte Typen {unbekannte_typen}")

# TL Categories
TOP_LEVEL_CATEGORIES = sorted(set(typ_to_category.values()))
print(f"Categories: {TOP_LEVEL_CATEGORIES}")

# strommix_gesamt[strommix_gesamt['Jahr'] == 2002].sort_values('Kategorie')
strommix_gesamt.head(20)



In [ ]:
# Check: Jährliche Summe der Elektrizitätserzeugung (netto) #### FIXME move or think about it, do we wanna show anything about this?, would be good tbh
jahres_summen = strommix_gesamt.groupby("Jahr")["Elektrizitätserzeugung (netto)"].sum().reset_index()

# MWh -> TWh 
jahres_summen["Elektrizität (TWh)"] = jahres_summen["Elektrizitätserzeugung (netto)"] / 1_000_000

jahres_summen = jahres_summen[["Jahr", "Elektrizität (TWh)"]].sort_values("Jahr")
display(jahres_summen)


In [ ]:
df_clean = strommix_gesamt[
    ~strommix_gesamt["Typ"].isin(["Einfuhr von Elektrizität", "Ausfuhr von Elektrizität"])
]

# Summe aller Typen (ohne Austauschsaldo)
produktion = df_clean[df_clean["Typ"] != "Austauschsaldo"]
produktion_summe = produktion.groupby("Jahr")["Elektrizitätserzeugung (netto)"].sum().reset_index()
produktion_summe = produktion_summe.rename(columns={"Elektrizitätserzeugung (netto)": "Inlandsproduktion"})

# Austauschsaldo (positiv = Import, negativ = Export)
saldo = df_clean[df_clean["Typ"] == "Austauschsaldo"][["Jahr", "Elektrizitätserzeugung (netto)"]]
saldo = saldo.rename(columns={"Elektrizitätserzeugung (netto)": "Austauschsaldo"})

# Zusammenführen
basis = produktion_summe.merge(saldo, on="Jahr", how="left").fillna(0)

# Berechne Kategorie-Anteile (inkl. Import!)
produktion_rel = produktion.merge(basis, on="Jahr")
produktion_rel["Anteil"] = produktion_rel["Elektrizitätserzeugung (netto)"] / (
    produktion_rel["Inlandsproduktion"] + produktion_rel["Austauschsaldo"].clip(lower=0)
)

# Gruppieren nach Kategorie
produktion_kat = produktion_rel.groupby(["Jahr", "Kategorie"])["Anteil"].sum().reset_index()

# Importanteil (innerhalb der 100%)
import_df = basis[basis["Austauschsaldo"] > 0].copy()
import_df["Anteil"] = import_df["Austauschsaldo"] / (import_df["Inlandsproduktion"] + import_df["Austauschsaldo"])
import_df["Kategorie"] = "Importanteil"

# Exportüberschuss (oberhalb der 100%)
export_df = basis[basis["Austauschsaldo"] < 0].copy()
export_df["Anteil"] = -export_df["Austauschsaldo"] / export_df["Inlandsproduktion"]
export_df["Kategorie"] = "Exportüberschuss"

# Zusammenführen für den Plot
df_plot_ready = pd.concat([
    produktion_kat[["Jahr", "Kategorie", "Anteil"]],
    import_df[["Jahr", "Kategorie", "Anteil"]],
    export_df[["Jahr", "Kategorie", "Anteil"]],
])

# Pivot + Sortierung: Exportüberschuss zuletzt
pivot_plot = df_plot_ready.pivot_table(
    index="Jahr",
    columns="Kategorie",
    values="Anteil",
    aggfunc="sum"
).fillna(0)

# Export & Import ans Ende
cols = list(pivot_plot.columns)
for special in ["Importanteil", "Exportüberschuss"]:
    if special in cols:
        cols = [c for c in cols if c != special] + [special]
pivot_plot = pivot_plot[cols]

# Plot

tab20_colors = plt.get_cmap("tab20").colors
safe_colors = [c for i, c in enumerate(tab20_colors) if i not in [4,5,6]]
categories = list(pivot_plot.columns)

#['Abfall', 'Andere Erneuerbare', 'Biomasse', 'Braunkohle', 'Erdgas', 'Export', 'Geothermie', 'Import', 'Import/Export', 'Industriegas', 'Kernenergie', 'Mineralöl', 'SUMME', 'Solar', 'Sonstiges', 'Steinkohle', 'Wasserkraft', 'Wasserstoff', 'Wind']
color_map = {
    "Abfall": "darkslategray",
    "Biomasse": "forestgreen",
    "Braunkohle": "saddlebrown",
    "Erdgas": "lightskyblue",
    "Geothermie": "darkorange",
    "Industriegas": "lightgray",
    "Kernenergie": "mediumvioletred",
    "Mineralöl": "dimgray",
    "Solar": "gold",
    "Sonstiges": "olive",
    "Steinkohle": "black",
    "Wasserkraft": "royalblue",
    "Wind": "lightblue",
    "Wasserstoff": "cyan",
    "Andere Erneuerbare": "pink",
    "Importanteil": "red",
    "Exportüberschuss": "lime",
}

# Farben in der korrekten Reihenfolge als Liste
colors = [color_map[cat] for cat in categories]

pivot_plot.plot(
    kind="bar",
    stacked=True,
    figsize=(16, 8),
    color=colors
)

plt.axhline(1, color='black', linestyle='--', linewidth=1)  # 100%-Linie
plt.title("Strommix Deutschland nach Oberkategorien (mit Export / Import)", fontsize=15)
plt.ylabel("Anteil am Stromverbrauch", fontsize=12)
plt.xlabel("Jahr")
plt.legend(title="Kategorie", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()



In [ ]:
# Inlandsproduktion
produktion_only = produktion_kat.copy()
produktion_only = produktion_only[~produktion_only["Kategorie"].isin(["Importanteil", "Exportüberschuss"])]

# Normierung
produktion_only_pivot = produktion_only.pivot_table(
    index="Jahr",
    columns="Kategorie",
    values="Anteil",
    aggfunc="sum"
).fillna(0)
produktion_only_pivot = produktion_only_pivot.div(produktion_only_pivot.sum(axis=1), axis=0)

categories_no_import_export = list(produktion_only_pivot.columns)
colors_no_import_export = [color_map[cat] for cat in categories_no_import_export]

produktion_only_pivot.plot(
    kind="bar",
    stacked=True,
    figsize=(16, 8),
    color=colors_no_import_export
)

plt.axhline(1, color='black', linestyle='--', linewidth=1)  # 100%
plt.title("Stromerzeugung Deutschland nach Oberkategorien (ohne Import/Export)", fontsize=15)
plt.ylabel("Anteil an der Inlandsproduktion", fontsize=12)
plt.xlabel("Jahr")
plt.legend(title="Kategorie", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


In [ ]:
# Nur Inlandsproduktion, ohne Import/Export
produktion_absolute_only = produktion.copy()
produktion_absolute_only = produktion_absolute_only[
    ~produktion_absolute_only["Kategorie"].isin(["Importanteil", "Exportüberschuss"])
]

# Pivot auf absolute Werte
produktion_absolute_pivot = produktion_absolute_only.pivot_table(
    index="Jahr",
    columns="Kategorie",
    values="Elektrizitätserzeugung (netto)",
    aggfunc="sum"
).fillna(0)

# Einheit umrechnen: MWh -> TWh
produktion_absolute_pivot = produktion_absolute_pivot / 1_000_000  # 1 TWh = 1 Mio MWh

# Farben in korrekter Reihenfolge
categories_absolute = list(produktion_absolute_pivot.columns)
colors_absolute = [color_map.get(cat, "hotpink") for cat in categories_absolute]

# Plot
ax = produktion_absolute_pivot.plot(
    kind="bar",
    stacked=True,
    figsize=(16, 8),
    color=colors_absolute
)

ax.set_title("Stromerzeugung Deutschland – Absolute Werte nach Oberkategorien (ohne Import/Export)", fontsize=15)
ax.set_ylabel("Stromerzeugung [TWh]", fontsize=12)
ax.set_xlabel("Jahr")
ax.legend(title="Kategorie", bbox_to_anchor=(1.02, 1), loc="upper left")
ax.grid(axis='y', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()


In [ ]:
#['Abfall', 'Andere Erneuerbare', 'Biomasse', 'Braunkohle', 'Erdgas', 'Export', 'Geothermie', 'Import', 'Import/Export', 'Industriegas', 'Kernenergie', 'Mineralöl', 
# 'SUMME', 'Solar', 'Sonstiges', 'Steinkohle', 'Wasserkraft', 'Wasserstoff', 'Wind']
include_kategorien = ['Abfall', 'Biomasse', 'Braunkohle', 'Erdgas', 
                      'Geothermie', 'Industriegas', 'Kernenergie', 
                      'Mineralöl', 'Solar', 'Sonstiges', 'Steinkohle', 
                      'Wasserkraft', 'Wind', 'Wasserstoff', 'Andere Erneuerbare'] # 'Import', 'Export'

print(list(set(include_kategorien).symmetric_difference(TOP_LEVEL_CATEGORIES)))

strommix_filtered = strommix_gesamt[strommix_gesamt["Kategorie"].isin(include_kategorien)]

# Sum by Kategorie and Jahr
strommix_for_prophet = (
    strommix_filtered
    .groupby(["Kategorie", "Jahr"], as_index=False)
    .agg({"Elektrizitätserzeugung (netto)": "sum"})
)

# Prophet labels
strommix_for_prophet = strommix_for_prophet.rename(columns={
    "Jahr": "ds",
    "Elektrizitätserzeugung (netto)": "y"
})

# year to datetime
strommix_for_prophet["ds"] = pd.to_datetime(strommix_for_prophet["ds"], format="%Y")

# Inbetween to print the included categories in the data
included_categories_test = sorted(strommix_for_prophet["Kategorie"].unique())
print(f"Prophet Kategorien: {', '.join(included_categories_test)}")
ignorierte_kategorien = [cat for cat in TOP_LEVEL_CATEGORIES if cat not in included_categories_test]

print(f"Ignorierte Kategorien: {', '.join(ignorierte_kategorien)}")
print(f"Es sind {len(included_categories_test)} from {len(include_kategorien)} that are allowed from {len(TOP_LEVEL_CATEGORIES)} total.")


strommix_for_prophet


In [ ]:
from prophet import Prophet

forecast_years = [2025, 2030, 2035, 2040, 2045, 2050, 2055, 2060]
future_dates = pd.DataFrame({'ds': pd.to_datetime([f'{year}-01-01' for year in forecast_years])})

all_forecasts = []

for category in strommix_for_prophet['Kategorie'].unique():
    # Filter data for the current category
    df_cat = strommix_for_prophet[strommix_for_prophet['Kategorie'] == category][['ds', 'y']].copy()
    df_cat['ds'] = pd.to_datetime(df_cat['ds'])

    # Skip if all values are zero or missing
    if df_cat['y'].sum() == 0 or df_cat['y'].isna().all():
        continue

    model = Prophet(yearly_seasonality=False, daily_seasonality=False, weekly_seasonality=False)
    model.fit(df_cat)

    # Predict for future years
    forecast = model.predict(future_dates)
    forecast['Kategorie'] = category
    
        
    # Clamp forecasts to zero to avoid negative energy values
    forecast['yhat'] = forecast['yhat'].clip(lower=0)  
  

    forecast_result = forecast[['ds', 'yhat', 'Kategorie']]

    all_forecasts.append(forecast_result)

# Combine all category forecasts
final_forecast_df = pd.concat(all_forecasts, ignore_index=True)


In [ ]:
# Modify Predictions

# Clamp Kernenergie to 0 because it has been banned
final_forecast_df.loc[final_forecast_df["Kategorie"] == "Kernenergie", "yhat"] = 0


In [ ]:
import matplotlib.pyplot as plt

# Merge original and forecasted data for unified plotting
original = strommix_for_prophet.copy()
original['ds'] = pd.to_datetime(original['ds'])
original['Type'] = 'Historisch'
original = original.rename(columns={'y': 'value'})

forecast = final_forecast_df.copy()
forecast['Type'] = 'Prognose'
forecast = forecast.rename(columns={'yhat': 'value'})

# Combine both
combined = pd.concat([
    original[['Kategorie', 'ds', 'value', 'Type']],
    forecast[['Kategorie', 'ds', 'value', 'Type']]
])


combined[combined['ds'] == '2025-01-01']

In [ ]:
# Convert to TWh
combined_twh = combined.copy()
combined_twh["value"] = combined_twh["value"] / 1_000_000

# Get global y-axis limits for consistent scaling
y_min = combined_twh['value'].min()
y_max = combined_twh['value'].max()

# Plot and save each category
for category in combined_twh['Kategorie'].unique():
    cat_data = combined_twh[combined_twh['Kategorie'] == category]
    fig, ax = plt.subplots(figsize=(10, 6))

    for label, grp in cat_data.groupby('Type'):
        marker = 'o' if label == 'Historisch' else 'x'
        ax.plot(grp['ds'], grp['value'], marker=marker, label=label)

    ax.set_title(category)
    ax.set_xlabel('Jahr')
    ax.set_ylabel('Stromerzeugung (TWh)')
    # ax.set_ylim(y_min, y_max)
    ax.legend()
    ax.grid(True)

    # Save the plot
    cat_name = category.lower().replace(' ', '_').replace('/', "_")
    filename = f"prediction_{cat_name}.png"
    fig.savefig(os.path.join(PLOT_PREDICTIONS_DIR, filename), dpi=300, bbox_inches='tight')

    # Show in notebook
    plt.show()


### Scenario 1: Literature Data with filled in predictions

#### Long Format

In [ ]:
import pandas as pd

CATEGORY_BLACKLIST = ["wind_onshore", "wind_offshore"]
COLUMN_BLACKLIST = ["Notes", "Category"]

# Load Excel file, assuming your data is in the first sheet
df_literature_short = pd.read_excel("in/germany_gridmix_for_Premise.xlsx", sheet_name="DATA_SHORT")

df_literature_short = df_literature_short[
    ~df_literature_short["Label"].isin(CATEGORY_BLACKLIST)
    # ~df_lit["Label" == "wind_offshore"] 
]

df_literature_short[df_literature_short.select_dtypes(include='number').columns] /= 100
df_literature_short = df_literature_short.drop(columns=["Notes", "Category"])

# Preview
print("Short")
print(df_literature_short.head(20))

df_literature_long = pd.read_excel("in/germany_gridmix_for_Premise.xlsx", sheet_name="DATA_LONG")

df_literature_long = df_literature_long[
    ~df_literature_long["Label"].isin(["wind_onshore", "wind_offshore"])
]

df_literature_long = df_literature_long.drop(columns=["Notes", "Category"])
df_literature_long[df_literature_long.select_dtypes(include='number').columns] /= 100

# Preview
print("\n\nlong")
print(df_literature_long)

print("SHORT total per year:\n", df_literature_short.select_dtypes(include='number').sum())
print("\nLONG total per year:\n", df_literature_long.select_dtypes(include='number').sum())



In [ ]:
# Step 1: Extract year and normalize predicted absolute values into shares
df_combined = combined.copy()

print("\n[INFO] Step 1: Normalize predicted values to shares")
df_combined["Year"] = pd.to_datetime(df_combined["ds"]).dt.year
df_combined["year_total"] = df_combined.groupby("Year")["value"].transform("sum")
df_combined["predicted_share"] = df_combined["value"] / df_combined["year_total"]

print("[DEBUG] Sample of predicted shares:")
print(df_combined[["Kategorie", "Year", "value", "predicted_share"]].head())

# Step 2: Convert df_literature_short to long format
print("\n[INFO] Step 2: Reshape df_literature_short to long format")
df_short_long = df_literature_short.melt(
    id_vars=["Label", "Energy Source", "Unit"],
    var_name="Year",
    value_name="observed_share"
)
df_short_long["Year"] = df_short_long["Year"].astype(int)

print("[DEBUG] Sample of observed shares (long format):")
print(df_short_long.head())

# Step 3: Merge observed and predicted values using only 'Energy Source'
print("\n[INFO] Step 3: Merge observed and predicted shares on 'Energy Source' only")
merged = pd.merge(
    df_short_long,
    df_combined[["Kategorie", "Year", "predicted_share"]],
    how="left",
    left_on=["Energy Source", "Year"],
    right_on=["Kategorie", "Year"]
)

print("[DEBUG] Merged result:")
print(merged.head(10))

# Step 4: Strictly prefer observed (even if 0.0), fallback to predicted only if NaN
print("\n[INFO] Step 4: Combine observed and predicted shares (no renormalization)")
merged["use_predicted"] = merged["observed_share"].isna()
merged["final_share"] = merged.apply(
    lambda row: row["predicted_share"] if row["use_predicted"] else row["observed_share"],
    axis=1
)


# Step 4.5: Compare predicted vs observed for debugging
print("\n[DEBUG] Step 4.5: Differences between observed and predicted shares (where both exist)")
diff_check = merged[
    merged["observed_share"].notna() & merged["predicted_share"].notna()
].copy()
diff_check["abs_diff"] = (diff_check["observed_share"] - diff_check["predicted_share"]).abs()
print(diff_check[["Energy Source", "Year", "observed_share", "predicted_share", "abs_diff"]].sort_values(by="abs_diff", ascending=False))


# Step 5: Compute sum of shares per year (for coverage insight)
print("\n[INFO] Step 5: Compute yearly share completeness (sums < 1.0 = missing technologies)")
merged["sum_per_year"] = merged.groupby("Year")["final_share"].transform("sum")

# Debug print for one year
debug_year = 2030
print(f"\n[DEBUG] Shares in {debug_year} and total sum:")
print(merged[merged["Year"] == debug_year][['Energy Source', 'final_share']].sort_values(by="final_share", ascending=False))
print(f"Total share in {debug_year}: {merged[merged['Year'] == debug_year]['final_share'].sum():.4f}")

# Step 6: Pivot back to wide format
print("\n[INFO] Step 6: Pivoting back to wide format without renormalization")
final_df = merged.pivot_table(
    index=["Energy Source", "Unit"],
    columns="Year",
    values="final_share"
).reset_index()

print("[DEBUG] Final mixed table (not normalized to 1.0):")
print(final_df.head())

# Step 7: Print final coverage per year
print("\n[INFO] Total share per year (should be <= 1.0):")
print(final_df.select_dtypes(include='number').sum())

print("[DEBUG] Any unexpected technologies?")
print(merged[merged["Kategorie"].notna() & merged["Energy Source"].isna()])


In [ ]:
# fixme may not work, tries to renomalize the shares
# Step 1: Extract year from combined and normalize values to predicted shares
print("\n[INFO] Step 1: Normalize predicted values to shares")
combined["Year"] = pd.to_datetime(combined["ds"]).dt.year
combined["year_total"] = combined.groupby("Year")["value"].transform("sum")
combined["predicted_share"] = combined["value"] / combined["year_total"]

# Display debug information for predicted shares
print("[DEBUG] Sample of predicted shares (normalized):")
print(combined[["Kategorie", "Year", "value", "predicted_share"]].head())

# Step 2: Convert df_literature_short from wide to long format
print("\n[INFO] Step 2: Reshape df_literature_short to long format")
df_short_long = df_literature_short.melt(
    id_vars=["Label", "Energy Source", "Unit"],
    var_name="Year",
    value_name="observed_share"
)
df_short_long["Year"] = df_short_long["Year"].astype(int)

print("[DEBUG] Sample of observed shares (long format):")
print(df_short_long.head())

# Step 3: Merge with combined on Label/Kategorie and Year
print("\n[INFO] Step 3: Merge observed and predicted data")
merged = pd.merge(
    df_short_long,
    combined[["Kategorie", "Year", "predicted_share"]],
    how="outer",
    left_on=["Label", "Year"],
    right_on=["Kategorie", "Year"]
)

print("[DEBUG] Merged data (with both observed and predicted):")
print(merged.head(10))

# Step 4: Combine: use observed if available, otherwise fallback to predicted
print("\n[INFO] Step 4: Mix observed and predicted shares")
merged["final_share"] = merged["observed_share"].combine_first(merged["predicted_share"])

# Step 5: Re-normalize shares per year
print("\n[INFO] Step 5: Renormalizing final shares per year (to sum up to 1.0)")
merged["sum_per_year"] = merged.groupby("Year")["final_share"].transform("sum")
merged["normalized_share"] = merged["final_share"] / merged["sum_per_year"]

# Print debug summary for a specific year to verify
debug_year = 2030
print(f"\n[DEBUG] Normalized shares for year {debug_year}:")
print(merged[merged["Year"] == debug_year][["Label", "normalized_share"]].sort_values(by="normalized_share", ascending=False))

# Step 6: Pivot back to wide format
print("\n[INFO] Step 6: Pivoting back to wide format for final DataFrame")
final_df = merged.pivot_table(
    index=["Label", "Energy Source", "Unit"],
    columns="Year",
    values="normalized_share"
).reset_index()

print("[DEBUG] Final mixed and normalized table:")
print(final_df.head(10))

# Optional: show column totals to verify normalization
print("\n[INFO] Final column-wise sum (should be ~1.0):")
print(final_df.select_dtypes(include='number').sum())


In [ ]:

# df_lit_filled
# Result: df_lit_filled now contains original literature data, 
# with missing percentages filled in from df_combined

# TOP_LEVEL_CATEGORIES

# combined


# Convert df_combined["ds"] to year
combined["Year"] = pd.to_datetime(combined["ds"]).dt.year

# Calculate total value per year for normalization
year_totals = combined.groupby("Year")["value"].transform("sum")

# Create a new column: fractional share
combined["fraction"] = combined["value"] / year_totals

# Now fill only empty (NaN) cells in df_literature_short with normalized values
# for _, row in combined.iterrows():
#     category = row["Kategorie"]
#     year = row["Year"]
#     fraction = row["fraction"]

#     if year in df_literature_short.columns:
#         mask = (df_literature_short["Label"] == category) & (df_literature_short[year].isna())
#         df_literature_short.loc[mask, year] = fraction

for _, row in combined.iterrows():
    category = row["Kategorie"]
    year = row["Year"]
    fraction = row["fraction"]

    if year in df_literature_short.columns:
        mask = (df_literature_short["Label"] == category) & (df_literature_short[year].isna())

        if mask.any():
            # Print info about what's being filled
            print(f"Filling missing value for '{category}' in {year}: {fraction:.4f}")
            df_literature_short.loc[mask, year] = fraction


# === Optional: final check ===
print(df_literature_short.head(10))
print("\nColumn sums per year:")
print(df_literature_short.select_dtypes(include='number').sum())


# Convert ds to year
# combined["Year"] = pd.to_datetime(combined["ds"]).dt.year

# # Fill values into df_literature_short (wide format)
# for _, row in combined.iterrows():
#     category = row["Kategorie"]
#     year = row["Year"]
#     value = row["value"]

#     if year in df_literature_short.columns:
#         # Only fill if the target cell is NaN
#         mask = (df_literature_short["Label"] == category) & (df_literature_short[year].isna())
#         df_literature_short.loc[mask, year] = value


# # Normalize all numeric columns to percentage (i.e., columns sum to 1.0)
# # Get only the year columns (exclude metadata)
# year_cols = df_literature_short.select_dtypes(include='number').columns

# # Normalize per column
# df_literature_short[year_cols] = df_literature_short[year_cols].div(
#     df_literature_short[year_cols].sum(), axis=1
# )

# # Optional preview
# print("SHORT total per year:\n", df_literature_short.select_dtypes(include='number').sum())

# df_literature_short.head(20)


# Plots

In [ ]:
target_years = [2005, 2010, 2015, 2020, 2021, 2022, 2023, 2024, 2025, 2030, 2035, 2040, 2045, 2050, 2055, 2060]

# Helper func
def shade_forecast_area(ax, forecast_start_year=2025, color='lightgray', alpha=0.3, label="Predicted"):
    """
    Shade the forecast area starting from forecast_start_year on a bar plot.
    
    Parameters:
    - ax: matplotlib Axes object
    - forecast_start_year: int, first year of forecast
    - color: background color
    - alpha: transparency
    - label: optional text label to display
    """

    bar_centers = []
    seen_x_positions = set()

    for patch in ax.patches:
        try:
            x = patch.get_x()
            width = patch.get_width()
            x_center = x + width / 2

            if x_center not in seen_x_positions:
                bar_centers.append(x_center)
                seen_x_positions.add(x_center)
        except AttributeError:
            # ignore polygons without get_x()
            continue

    # Map bar_centers to years
    try:
        bar_years = [int(label.get_text()) for label in ax.get_xticklabels()]
    except:
        bar_years = list(range(len(bar_centers)))

    year_to_center = dict(zip(bar_years, bar_centers))

    if forecast_start_year in year_to_center:
        start_forecast = year_to_center[forecast_start_year] - 0.5
        ax.axvspan(
            start_forecast, ax.get_xlim()[1],
            color=color, alpha=alpha, zorder=0
        )

        if label:
            ax.text(
                start_forecast + 0.5,
                ax.get_ylim()[1] * 0.95,
                label,
                verticalalignment='top',
                horizontalalignment='left',
                fontsize=16,
                color='black'
            )



## ---


# Prepare production data (historical + forecast)
prod_combined = combined[combined['Kategorie'].isin(include_kategorien)].copy()
prod_combined["Jahr"] = prod_combined["ds"].dt.year
prod_combined = prod_combined[["Jahr", "Kategorie", "value", "Type"]]

# Historical Import/Export (2002–2024)
saldo_df = strommix_gesamt[strommix_gesamt["Typ"] == "Austauschsaldo"].copy()
saldo_df = saldo_df[(saldo_df["Jahr"] >= 2002) & (saldo_df["Jahr"] <= 2024)]
saldo_df = saldo_df[["Jahr", "Elektrizitätserzeugung (netto)"]].rename(columns={"Elektrizitätserzeugung (netto)": "Austauschsaldo"})

# Inlandsproduktion for historical years
historical_prod = prod_combined[prod_combined["Type"] == "Historisch"]
inlandsproduktion = historical_prod.groupby("Jahr")["value"].sum().reset_index().rename(columns={"value": "Inlandsproduktion"})

# Basis historical = production + saldo
basis = inlandsproduktion.merge(saldo_df, on="Jahr", how="left").fillna(0)

# Anteil berechnen für historische Daten
historical_merged = historical_prod.merge(basis, on="Jahr", how="left")
historical_merged["Anteil"] = historical_merged["value"] / (
    historical_merged["Inlandsproduktion"] + historical_merged["Austauschsaldo"].clip(lower=0)
)

# Forecast Anteile berechnen (ohne Import/Export)
forecast = prod_combined[prod_combined["Type"] == "Prognose"].copy()
forecast_total = forecast.groupby("Jahr")["value"].sum().reset_index().rename(columns={"value": "Inlandsproduktion"})
forecast = forecast.merge(forecast_total, on="Jahr")
forecast["Anteil"] = forecast["value"] / forecast["Inlandsproduktion"]

# Import/Export (nur historische Jahre)
import_df = basis[basis["Austauschsaldo"] > 0].copy()
import_df["Anteil"] = import_df["Austauschsaldo"] / (import_df["Inlandsproduktion"] + import_df["Austauschsaldo"])
import_df["Kategorie"] = "Importanteil"

export_df = basis[basis["Austauschsaldo"] < 0].copy()
export_df["Anteil"] = -export_df["Austauschsaldo"] / export_df["Inlandsproduktion"]
export_df["Kategorie"] = "Exportüberschuss"

# Combine all + filter to target_years
final_rel = pd.concat([
    historical_merged[["Jahr", "Kategorie", "Anteil"]],
    forecast[["Jahr", "Kategorie", "Anteil"]],
    import_df[["Jahr", "Kategorie", "Anteil"]],
    export_df[["Jahr", "Kategorie", "Anteil"]]
])
final_rel = final_rel[final_rel["Jahr"].isin(target_years)]

# Pivot für Stacked Bar Plot
pivot_plot = final_rel.pivot_table(
    index="Jahr",
    columns="Kategorie",
    values="Anteil",
    aggfunc="sum"
).fillna(0)

# Export/Import at end
cols = list(pivot_plot.columns)
for special in ["Importanteil", "Exportüberschuss"]:
    if special in cols:
        cols = [c for c in cols if c != special] + [special]
pivot_plot = pivot_plot[cols]

# Farben und Plot
tab20_colors = plt.get_cmap("tab20").colors
safe_colors = [c for i, c in enumerate(tab20_colors) if i not in [4,5,6]]
categories = list(pivot_plot.columns)

colors = [color_map.get(cat, "hotpink") for cat in categories]


# Plot
ax = pivot_plot.plot(
    kind="bar",
    stacked=True,
    figsize=(16, 8),
    color=colors
)

plt.axhline(1, color='black', linestyle='--', linewidth=1)
plt.title("Strommix Deutschland – Anteile nach Kategorie (mit Import/export, inkl. Prognosen)", fontsize=15)
plt.ylabel("Anteil am Stromverbrauch (normiert)", fontsize=12)
plt.xlabel("Jahr")
plt.legend(title="Kategorie", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.grid(axis='y', linestyle='--', alpha=0.5)

shade_forecast_area(ax)


plt.tight_layout()
plt.show()


In [ ]:
# New pivot without Importanteil / Exportüberschuss
pivot_plot_no_import = final_rel[
    ~final_rel["Kategorie"].isin(["Importanteil", "Exportüberschuss"])
].pivot_table(
    index="Jahr",
    columns="Kategorie",
    values="Anteil",
    aggfunc="sum"
).fillna(0)

# Normalize again (sum = 1.0 each year)
pivot_plot_no_import = pivot_plot_no_import.div(pivot_plot_no_import.sum(axis=1), axis=0)

# New category list
categories_no_import = list(pivot_plot_no_import.columns)

# Reuse color_map safely
colors_no_import = [color_map.get(cat, "hotpink") for cat in categories_no_import]

# Plot (no Import/Export)
ax = pivot_plot_no_import.plot(
    kind="bar",
    stacked=True,
    figsize=(16, 8),
    color=colors_no_import
)

plt.axhline(1, color='black', linestyle='--', linewidth=1)
plt.title("Strommix Deutschland – Anteile nach Kategorie (ohne Import/Export, inkl. Prognosen)", fontsize=15)
plt.ylabel("Anteil an der Inlandsproduktion (normiert)", fontsize=12)
plt.xlabel("Jahr")
plt.legend(title="Kategorie", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.grid(axis='y', linestyle='--', alpha=0.5)

shade_forecast_area(ax) 

plt.savefig(os.path.join(PLOT_DIR, "strommix_prognosen.png"), dpi=300, bbox_inches="tight")


plt.tight_layout()
plt.show()


In [ ]:
# Only use categories 
prod_absolute = combined[combined['Kategorie'].isin(include_kategorien)].copy()

# Convert MWh to TWh
prod_absolute["value"] = prod_absolute["value"] / 1_000_000  # MWh -> TWh

# Prepare absolute pivot table (no Import/Export)
prod_absolute["Jahr"] = prod_absolute["ds"].dt.year

pivot_absolute = prod_absolute[prod_absolute["Jahr"].isin(target_years)].pivot_table(
    index="Jahr",
    columns="Kategorie",
    values="value",
    aggfunc="sum"
).fillna(0)

# Sort years manually to match target_years
pivot_absolute = pivot_absolute.reindex(target_years)

# Define color order
categories_absolute = list(pivot_absolute.columns)
colors_absolute = [color_map.get(cat, "hotpink") for cat in categories_absolute]

# Create plot and capture ax
ax = pivot_absolute.plot(
    kind="bar",
    stacked=True,
    figsize=(16, 8),
    color=colors_absolute
)

# Standard labels
ax.set_title("Strommix Deutschland – Absolute Stromerzeugung in TWh (ohne Import/Export, inkl. Prognosen)", fontsize=15)
ax.set_ylabel("Stromerzeugung [TWh]", fontsize=12)
ax.set_xlabel("Jahr")
ax.legend(title="Kategorie", bbox_to_anchor=(1.02, 1), loc="upper left")
ax.grid(axis='y', linestyle='--', alpha=0.5)

shade_forecast_area(ax)

plt.savefig(os.path.join(PLOT_DIR, "strommix_prediction_abs.png"), dpi=300, bbox_inches="tight")

plt.tight_layout()
plt.show()


In [ ]:

# https://premise.readthedocs.io/en/latest/mapping.html
# MAPPINGS: https://github.com/polca/premise/blob/master/premise/iam_variables_mapping/electricity_variables.yaml

# Export

In [ ]:
# Strommix ursprungliche Daten Export, ohne Prognose, Excel


# Pivot: Jahre = Zeilen, Kategorien = Spalten
strommix_wide = strommix_gesamt.pivot_table(
    index=["Kategorie", "Typ"],
    columns="Jahr",
    values="Elektrizitätserzeugung (netto)",
).fillna(0)

strommix_wide.insert(0, "Unit", "MWh")

# Index resetten für Export
strommix_wide = strommix_wide.reset_index()
with pd.ExcelWriter("strommix_formatiert.xlsx", engine='xlsxwriter') as writer:
    strommix_wide.to_excel(writer, sheet_name='Strommix', index=False)

    workbook  = writer.book
    worksheet = writer.sheets['Strommix']

    # 1st column (JAHR)
    worksheet.set_column(0, 3, 16, workbook.add_format({
        'bold': True,
        'align': 'left'
    }))

    # Data columns
    worksheet.set_column(3, strommix_wide.shape[1], 20, workbook.add_format({
        'align': 'left'
    }))

    # Header formattierung
    header_format = workbook.add_format({'bold': True, 'bg_color': '#D9D9D9'})
    worksheet.set_row(0, 16, header_format)



strommix_wide.head(20)


In [ ]:
# Export predictions for Premise, scenario 2, all predicted data

# CONFIG
# kategorie_to_variable = {
#     "Abfall": "Electricity generation|Wastes",  # Aggregated
#     "Biomasse": "Electricity generation|Biomass",  # Top-level
#     "Braunkohle": "Electricity generation|Coal|Fossil|Lignite",  
#     "Erdgas": "Electricity generation|Gas|Fossil gases",  
#     "Geothermie": "Electricity generation|Geothermal",  
#     "Industriegas": "Electricity generation|Gas|Fossil gases|Industrial CHPs gas",  
#     "Kernenergie": "Electricity generation|Nuclear Fuel|Nuclear power plants",
#     "Mineralöl": "Electricity generation|Oil|Fossil liquids",  
#     "Solar": "Electricity generation|Solar",  
#     "Sonstiges": "Electricity generation|Other",  
#     "Steinkohle": "Electricity generation|Coal|Fossil|Hard coal",  
#     "Wasserkraft": "Electricity generation|Hydro",  
#     "Wind": "Electricity generation|Wind" ,
#     "Wasserstoff": "Electricity generation|Hydrogen", # todo to be added
# }

kategorie_to_variable = {
    "Abfall": "Electricity|Production|Waste",                    # closest fit
    "Biomasse": "Electricity|Production|Biomass",
    "Braunkohle": "Electricity|Production|Coal",                 # REMIND doesn't split hard/lignite
    "Steinkohle": "Electricity|Production|Coal",
    "Erdgas": "Electricity|Production|Gas",
    "Industriegas": "Electricity|Production|Gas",                # industrial CHP gas assumed to feed into grid
    "Kernenergie": "Electricity|Production|Nuclear",
    "Mineralöl": "Electricity|Production|Oil",
    "Solar": "Electricity|Production|Solar",
    "Wind": "Electricity|Production|Wind",
    "Wasserkraft": "Electricity|Production|Hydro",
    "Geothermie": "Electricity|Production|Geothermal",
    "Wasserstoff": "Electricity|Production|Hydrogen",           # not in REMIND by default, but safe fallback
    "Sonstiges": "Electricity|Production|Other"                 # catch-all (maps to synthetic or unspecified)
}


target_years = [2005, 2010, 2015, 2020, 2025, 2030, 2035, 2040, 2045, 2050, 2055, 2060]

# Step 1: Filter combined dataset
df_filtered = combined[combined['ds'].dt.year.isin(target_years)].copy()

# Convert MWh to PJ
df_filtered['PJ'] = df_filtered['value'] * 3.6 / 1_000_000
df_filtered['year'] = df_filtered['ds'].dt.year

# Map Kategorie -> IAM variable name
df_filtered['variable'] = df_filtered['Kategorie'].map(kategorie_to_variable)

# Drop rows with unmapped variables (optional safety)
df_filtered = df_filtered.dropna(subset=['variable'])

# Step 2: Pivot to wide format
pivot = df_filtered.pivot_table(
    index="variable",
    columns="year",
    values="PJ",
    aggfunc="sum"
).reset_index()

# Add SWEET/Premise metadata columns
pivot["scenario"] = "Base"
pivot["region"] = "DE"
pivot["unit"] = "PJ/yr."
pivot["model"] = "DE_GRIDMIX_PREDICTED"

# Reorder columns to match SWEET structure
export_columns = ["model", "scenario", "region", "variable", "unit"] + target_years
final_export = pivot[export_columns]

# Export to CSV
final_export.to_csv(f"{OUTPUT_DIR}/DE_GRIDMIX_SCENARIO_02_PREDICTIONS.csv", index=False)


final_export.head(20)